In [1]:
from importlib import reload
import fitsio as fio
import numpy as np
import pandas as pd

import healpy as hp
import copy
import sys
import glob
import os
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn.decomposition as decomp
import skysampler.emulator as emulator
import skysampler.indexer as indexer

import matplotlib as mpl
import subprocess as sp
import scipy.interpolate as interpolate
import pickle as pickle

import multiprocessing as mp

import skysampler.utils as utils


config read from file: /home/moon/vargatn/DES/Y3_WORK/skysampler-config/config/desy3-gamma-wide-config.yaml


In [2]:
def result_reader2(samples, scores, m_factor=20, nratio=1., seed=None):
    
    dc_score = np.exp(scores["dc"]) * np.abs(scores["dc_jac"])
    wr_score = np.exp(scores["wr"]) * np.abs(scores["wr_jac"])
    wcr_clust_score = np.exp(scores["wcr_clust"]) * np.abs(scores["wcr_clust_jac"])
    wcr_rands_score = np.exp(scores["wcr_rands"]) * np.abs(scores["wcr_rands_jac"])
#     fcls = fclfunc(samples["LOGR"]) - 1.
    
#     umult = np.max((fcls, np.ones(len(fcls))), axis=0)
#     print(umult)
    rng = np.random.RandomState(seed)
    uniform = rng.uniform(0, 1, len(samples)) #* umult
    
    p_proposal = m_factor * dc_score * wr_score
    p_rands_ref = wcr_rands_score
    p_clust_ref = wcr_clust_score #* fcls
    print(nratio)
    inds_field = (uniform < (p_rands_ref / nratio / p_proposal))
    inds_clust = ((p_rands_ref / nratio / p_proposal) < uniform) * (uniform < (p_clust_ref  / p_proposal))
    inds_2d = ((uniform < (p_clust_ref  / p_proposal)))

    return inds_field, inds_clust, inds_2d

In [3]:

tag_root = "des-dc2-alpha_concentric_sample-v02_test-01"
NREPEATS = 4
NSAMPLES = 1600000
NCHUNKS = 160
bandwidth=0.1

root_path = "/home/moon/vargatn/DES/PROJECTS/LSST/DATA/runs/"
deep_data_path = "/home/moon/vargatn/DES/PROJECTS/LSST/DATA/cosmoDC2_v1.1.4_refpixel_morepars-v2-8786.h5"
wide_clust_data_path = "/home/moon/vargatn/DES/PROJECTS/LSST/DATA/multi-indexer-epsilon_narrow-z_zoom_high-l_v004_clust_z0_l1_dict.p"
wide_rands_data_path = "/home/moon/vargatn/DES/PROJECTS/LSST/DATA/multi-indexer-epsilon_narrow-z_zoom_high-l_v004_rands_z0_l1_dict.p"

LOGR_DRAW_RMINS = np.array([-3, -0.5, 0., 0.5])
LOGR_DRAW_RMAXS = np.array([-0.5, 0., 0.5, 1.2])
LOGR_CAT_RMAXS = [0., 0.5, 1.1, 1.2]

deep_c_settings = {
    "columns": [
        ("MAG_I", "mag_i"),
        ("COLOR_G_R", ("mag_g", "mag_r", "-")),
        ("COLOR_R_I", ("mag_r", "mag_i", "-")),
    ],
    "logs": [False, False, False, False],
    "limits": [(17, 22.5), (-1, 3), (-1, 3), (-1, 3)],
}
deep_smc_settings = {
    "columns": [
        ("GABS", ("ellipticity_1_true", "ellipticity_2_true", "SQSUM")),
        ("SIZE", "size_true"),
        ("SIZE_MINOR", "size_minor_true"),
        ("SIZE_BULGE", "size_bulge_true"),
        ("SIZE_MINOR_BULGE", "size_minor_bulge_true"),
        ("SIZE_DISK", "size_disk_true"),
        ("SIZE_MINOR_DISK", "size_minor_disk_true"),
        ("MAG_I", "mag_i"),
        ("COLOR_G_R", ("mag_g", "mag_r", "-")),
        ("COLOR_R_I", ("mag_r", "mag_i", "-")),
        ("COLOR_I_Z", ("mag_i", "mag_z", "-")),
        ("STELLAR_MASS", "stellar_mass"),
    ],
    "logs": [False, True, True, True, True, True, True, False, False, False, False, True],
    "limits": [(0., 1.), (-1, 5), (-1, 5), (-1, 5), (-1, 5), (-1, 5), (-1, 5), (17, 25), (-1, 3), (-1, 3), (-1, 3), (10**3, 10**13)],
}

wide_cr_settings = {
    "columns": [
        ("MAG_I", "MOF_CM_MAG_CORRECTED_I"),
        ("COLOR_G_R", ("MOF_CM_MAG_CORRECTED_G", "MOF_CM_MAG_CORRECTED_R", "-")),
        ("COLOR_R_I", ("MOF_CM_MAG_CORRECTED_R", "MOF_CM_MAG_CORRECTED_I", "-")),
        ("LOGR", "DIST"),
    ],
    "logs": [False, False, False, True],
    "limits": [(17, 22.5), (-1, 3), (-1, 3), (1e-3, 50.), ],
}
wide_r_settings = {
    "columns": [
        ("MAG_I", "MOF_CM_MAG_CORRECTED_I"),
        ("LOGR", "DIST"),
    ],
    "logs": [False, True,],
    "limits": [(17, 22.5), (1e-3, 50.),],
}
columns = {
    "cols_dc": ["COLOR_G_R", "COLOR_R_I",],
    "cols_wr": ["LOGR",],
    "cols_wcr": ["COLOR_G_R", "COLOR_R_I", "LOGR",],
}

In [4]:
y3_ctab = pd.read_hdf("/home/moon/vargatn/DES/PROJECTS/LSST/DATA/multi-indexer-epsilon_narrow-z_zoom_high-l_v004_clust_z0_l1.h5")

Note: detected 160 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
Note: NumExpr detected 160 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.


In [5]:
y3_rtab = pd.read_hdf("/home/moon/vargatn/DES/PROJECTS/LSST/DATA/multi-indexer-epsilon_narrow-z_zoom_high-l_v004_rands_z0_l1.h5")

In [6]:
print(wide_clust_data_path)
tmp = pickle.load(open(wide_clust_data_path, "rb"), encoding="latin1")
mdl_clust = indexer.IndexedDataContainer(**tmp)

print(wide_rands_data_path)
tmp = pickle.load(open(wide_rands_data_path, "rb"), encoding="latin1")
mdl_rands = indexer.IndexedDataContainer(**tmp)

print(deep_data_path)
refpixel = pd.read_hdf(deep_data_path, key="data")


/home/moon/vargatn/DES/PROJECTS/LSST/DATA/multi-indexer-epsilon_narrow-z_zoom_high-l_v004_clust_z0_l1_dict.p
/home/moon/vargatn/DES/PROJECTS/LSST/DATA/multi-indexer-epsilon_narrow-z_zoom_high-l_v004_rands_z0_l1_dict.p
/home/moon/vargatn/DES/PROJECTS/LSST/DATA/cosmoDC2_v1.1.4_refpixel_morepars-v2-8786.h5


In [7]:
tmp_wide_cr_settings = wide_cr_settings.copy()
_wide_cr_settings_rands = emulator.construct_wide_container(mdl_rands, tmp_wide_cr_settings, seed=1, drop="MAG_I")

# loading deep catalogs
_deep_c_settings = emulator.construct_deep_container(refpixel, deep_c_settings, seed=2, drop="MAG_I")
_deep_smc_settings = emulator.construct_deep_container(refpixel, deep_smc_settings, seed=3)

# loading cluster data
tmp_wide_cr_settings = wide_cr_settings.copy()
#             tmp_wide_cr_settings["limits"][-1] = (10**-3, 10**LOGR_CAT_RMAXS[i])
_wide_cr_settings_clust = emulator.construct_wide_container(mdl_clust, tmp_wide_cr_settings, seed=4, drop="MAG_I")

/home/moon/vargatn/DES/PROJECTS/skysampler/skysampler/emulator.py:158: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.alldata = pd.concat(np.array(self.samples)[valid_elements]).reset_index(drop=True)


(216060, 4)
(216060,)
(70310, 2)
(70310,)
(501640, 12)
(501640,)
(208230, 4)
(208230,)


In [8]:
hybrid_samples = []
hybrid_scores = []
for rbin in np.arange(4):
    print(rbin)
#     expr = "/e/ocean1/users/vargatn/EMULATOR/EPSILON/resamples/epsilon_concentric_sample_v06_run*_rbin" + str(rbin) + "*samples.fits" 
    expr = "/home/moon/vargatn/DES/PROJECTS/LSST/DATA/runs/des-dc2-alpha_concentric_sample-v02_test-01/des-dc2-alpha_concentric_sample-v02_test-01_run*_rbin{}*samples.fits".format(str(rbin))


    fnames_samples = np.sort(glob.glob(expr))
    fnames_scores = []
    for fname in fnames_samples:
        fnames_scores.append(fname.replace("samples.fits", "scores.fits"))

    samples_sep = []
    scores_sep = []
    for i, fname in enumerate(fnames_samples):
#         print(fname)
        samples_sep.append(fio.read(fname))
        scores_sep.append(fio.read(fnames_scores[i]))
        
    hybrid_samples.append(np.hstack(samples_sep))
    hybrid_scores.append(np.hstack(scores_sep))

0
1
2
3


In [9]:
mag_lims = (17, 22.5)
r_lims_all = [(-1.5, -0.5), (-0.5, 0.), (0, 0.5), (0.5, 1.0)]
redges = [-1.5, -0.5, 0., 0.5, 1.0]
rareas = np.array([np.pi*((10**redges[i+1])**2. - (10**redges[i])**2.) for i in np.arange(len(redges)-1)])

In [10]:
nc = 123
nr = 500

In [11]:
magcol = "MAG_I"
ii = ((y3_ctab[magcol] > mag_lims[0]) & (y3_ctab[magcol] < mag_lims[1]))
clust_los_nums = np.histogram(y3_ctab[ii]["LOGR"], weights=y3_ctab[ii]["WEIGHTS"], bins=redges)[0] / nc
ii = ((y3_rtab[magcol] > mag_lims[0]) & (y3_rtab[magcol] < mag_lims[1]))
rands_los_nums = np.histogram(y3_rtab[ii]["LOGR"], weights=y3_rtab[ii]["WEIGHTS"], bins=redges)[0] / nr
y3_nratios = clust_los_nums / rands_los_nums

In [12]:
hybrid_ifields2 = []
hybrid_iclusts2 = []
hybrid_i2ds2 = []
for rbin in np.arange(4):
    print(rbin)
    _ifield, _iclust, _i2d = result_reader2(hybrid_samples[rbin], hybrid_scores[rbin], nratio=y3_nratios[rbin], m_factor=100, seed=rbin)
    hybrid_ifields2.append(_ifield)
    hybrid_iclusts2.append(_iclust)
    hybrid_i2ds2.append(_i2d)

0
3.7455659109436623
1
2.4819160870030195
2
1.3948185773196122
3
1.0611671167284917


In [13]:
hybrid_csamples = []
for rbin in np.arange(4):
    print(rbin)
    tab = pd.DataFrame.from_records(hybrid_samples[rbin][hybrid_iclusts2[rbin]].byteswap().newbyteorder())
    tab.drop('index', axis=1, inplace=True)
    kde = emulator.KDEContainer(tab)
    kde.standardize_data()
    kde.construct_kde(0.01)
    _csample = kde.random_draw(4e6)
    hybrid_csamples.append(_csample)

0
1
2
3


In [14]:
hybrid_rsamples = []
for rbin in np.arange(4):
    print(rbin)
    tab = pd.DataFrame.from_records(hybrid_samples[rbin][hybrid_ifields2[rbin]].byteswap().newbyteorder())
    tab.drop('index', axis=1, inplace=True)
    kde = emulator.KDEContainer(tab)
    kde.standardize_data()
    kde.construct_kde(0.01)
    _rsample = kde.random_draw(4e6)
    hybrid_rsamples.append(_rsample)

0
1
2
3


In [15]:
hybrid_allsamples = []
for rbin in np.arange(4):
    print(rbin)
    tab = pd.DataFrame.from_records(hybrid_samples[rbin][hybrid_i2ds2[rbin]].byteswap().newbyteorder())
    tab.drop('index', axis=1, inplace=True)
    kde = emulator.KDEContainer(tab)
    kde.standardize_data()
    kde.construct_kde(0.01)
    _sample = kde.random_draw(4e6)
    hybrid_allsamples.append(_sample)

0
1
2
3


In [17]:
bcgcat = mdl_clust.samples[0]

bcgtab = pd.DataFrame()
bcgtab["RA"] = [0,]
bcgtab["DEC"] = [0.,]
bcgtab["X"] = [0.,]
bcgtab["Y"] = [0.,]

_imag = 17.62
_gr = 1.38
_ri = 0.54
_iz = 0.37
bcgtab["MAG_G"] = [_gr + _ri + _imag,]
bcgtab["MAG_R"] = [_ri + _imag,]
bcgtab["MAG_I"] = [_imag,]
bcgtab["MAG_Z"] = [_imag - _iz,]
bcgtab["FRACDEV"] = [1.,]
bcgtab["TSIZE"] = [33.20,]

bcg_gabs = 0.14
# bcg_gabs = 0.07
angle = np.random.uniform(0, np.pi, size=1)

bcgtab["G1"] = -1. * bcg_gabs * np.cos(2 * angle)
bcgtab["G2"] = -1. * bcg_gabs * np.sin(2 * angle)
# bcgtab["G1"] = [0,]
# bcgtab["G2"] = [0,]
bcgtab["LOC"] = 1

bcgtab["Z"] = 0.325
bcgtab["shear"] = 0.
# bcgtab["shear2"] = 0.
# bcgtab["Z"]

bcgtab["FLUX_G"] = 10 ** (0.4 *(30. - bcgtab["MAG_G"]))
bcgtab["FLUX_R"] = 10 ** (0.4 *(30. - bcgtab["MAG_R"]))
bcgtab["FLUX_I"] = 10 ** (0.4 *(30. - bcgtab["MAG_I"]))
bcgtab["FLUX_Z"] = 10 ** (0.4 *(30. - bcgtab["MAG_Z"]))

In [62]:
gals

,COLOR_G_R,COLOR_I_Z,COLOR_R_I,GABS,LOGR,MAG_I,SIZE,SIZE_BULGE,SIZE_DISK,SIZE_MINOR,SIZE_MINOR_BULGE,SIZE_MINOR_DISK,STELLAR_MASS,Z
3214367,1.536802,0.329729,0.511463,0.014104,-0.565018,18.813550,-0.142989,-0.144123,-0.079340,-0.152745,-0.160880,-0.105685,11.145405,0.325
3375285,1.381213,0.301441,0.464830,0.059532,-0.605286,19.809881,-0.165175,-0.186236,-0.013111,-0.218608,-0.233215,-0.107522,10.496935,0.325
1153842,1.055742,0.161564,0.335835,0.031551,-0.566649,21.960173,-0.360137,-0.488826,-0.158518,-0.389342,-0.489438,-0.199872,9.303776,0.325
3261377,1.340046,0.169480,0.388547,0.418884,-1.084094,22.361625,-0.575251,-0.824194,-0.583455,-0.963365,-0.864020,-0.964242,9.116773,0.325
1947103,1.648387,0.774728,1.038359,0.225303,-0.508000,21.679242,-0.234666,-0.230467,-0.129307,-0.432516,-0.439854,-0.392179,11.364313,0.325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3245737,0.962185,0.128482,0.250445,0.087588,0.723338,23.979161,-0.951189,-1.040071,-0.763119,-1.027391,-1.044389,-1.034336,7.706235,0.325
2347330,1.168025,0.118632,0.244377,0.234383,0.804540,24.428437,-0.925445,-1.165018,-0.924933,-1.135317,-1.214350,-1.137436,8.321273,0.325
1524822,0.358142,0.557981,0.427179,0.156079,0.917547,24.380021,-0.481959,-0.758426,-0.448963,-0.621514,-0.804891,-0.602016,8.833749,0.325
117759,0.341704,0.688356,0.313678,0.085997,0.989302,23.836259,-0.492849,-0.679169,-0.422398,-0.566616,-0.683296,-0.541674,9.869985,0.325


In [24]:
ctab = y3_ctab
rtab = y3_rtab
# print(n)
magcol = "MAG_I"
rcol = "LOGR"
mag_lims = (17, 22.5)
medges = np.linspace(17, 22.5, 6)
medges2 = np.linspace(22.5, 24.5, 4)

mag_slices = [(17, 19), (19, 20), (20, 21), (21, 22), (22, 22.5)]

r_lims_all = [(-1.5, -0.5), (-0.5, 0.), (0, 0.5), (0.5, 1.0)]
redges = [-1.5, -0.5, 0., 0.5, 1.0]
rareas = [np.pi*((10**redges[i+1])**2. - (10**redges[i])**2.) for i in np.arange(len(redges)-1)]

# drawing line - of - sight surface number of objects in each bin
ii = ((ctab[magcol] > mag_lims[0]) & (ctab[magcol] < mag_lims[1]))
clust_los_nums = np.histogram(ctab[ii]["LOGR"], weights=ctab[ii]["WEIGHTS"], bins=redges)[0] / nc

ii = ((rtab[magcol] > mag_lims[0]) & (rtab[magcol] < mag_lims[1]))
rands_los_nums = np.histogram(rtab[ii]["LOGR"], weights=rtab[ii]["WEIGHTS"], bins=redges)[0] / nr

frac = 1. / (clust_los_nums / rands_los_nums)
fac = 1
clust_nums_to_draw = clust_los_nums * (1 - frac) * fac
rands_nums_to_draw = rands_los_nums * fac

maglim_max1 = 22.5
maglim_max2 = 24.5
clust_gals = []
field_gals = []
for rbin, r_lims in enumerate(r_lims_all[:4]):
    print(rbin, r_lims)

    #######################################
    sample = hybrid_csamples[rbin]

    ii = ((ctab[rcol] > r_lims[0]) & (ctab[rcol] < r_lims[1]))
    vals = np.histogram(ctab[ii][magcol], weights=ctab[ii]["WEIGHTS"], bins=medges)[0]
    vals = vals / vals.sum()

    for i in np.arange(len(medges) - 1):
        _n_to_draw = clust_nums_to_draw[rbin] * vals[i]
        n_to_draw = np.random.poisson(_n_to_draw)

        index = ((sample[magcol] > medges[i]) & (sample[magcol] < medges[i + 1]) &
                 (sample[rcol] > r_lims[0]) & (sample[rcol] < r_lims[1])).values.astype(bool)
        ii = np.random.randint(0, len(sample.iloc[index]), size=n_to_draw)
        tmp = sample.iloc[index].iloc[ii]
        if len(tmp):
            clust_gals.append(tmp)   

    ii = ((sample[magcol] > mag_lims[0]) & (sample[magcol] < maglim_max1) &
          (sample[rcol] > r_lims[0]) & (sample[rcol] < r_lims[1]))
    num1 = len(sample[ii])

    ii = ((sample[magcol] > maglim_max1) & (sample[magcol] < maglim_max2) &
          (sample[rcol] > r_lims[0]) & (sample[rcol] < r_lims[1]))
    num2 = len(sample[ii])
    fac_clust = num2 / num1

    vals = np.histogram(sample[magcol], bins=medges2)[0]
    vals = vals / vals.sum()
    for i in np.arange(len(medges2) - 1):
        _n_to_draw = clust_nums_to_draw[rbin] * vals[i] * fac_clust
        n_to_draw = np.random.poisson(_n_to_draw)

        index = ((sample[magcol] > medges2[i]) & (sample[magcol] < medges2[i + 1]) &
        (sample[rcol] > r_lims[0]) & (sample[rcol] < r_lims[1])).values.astype(bool)
        ii = np.random.randint(0, len(sample.iloc[index]), size=n_to_draw)
        tmp = sample.iloc[index].iloc[ii]
        if len(tmp):
            clust_gals.append(tmp)   



gals = pd.concat(clust_gals, sort=True)
gals["Z"] = 0.325



0 (-1.5, -0.5)
1 (-0.5, 0.0)
2 (0, 0.5)
3 (0.5, 1.0)


In [ ]:

bcgtab["MAG_G"] = [_gr + _ri + _imag,]
bcgtab["MAG_R"] = [_ri + _imag,]
bcgtab["MAG_I"] = [_imag,]
bcgtab["MAG_Z"] = [_imag - _iz,]
bcgtab["FRACDEV"] = [1.,]
bcgtab["TSIZE"] = [33.20,]

In [73]:
np.sqrt(33.2)

5.761944116355173

In [172]:
_imag = 17.62
_gr = 1.38
_ri = 0.54
_iz = 0.37
bcg_table = pd.DataFrame()
bcg_table["raJ2000"] = np.deg2rad(np.array([64,]))
bcg_table["decJ2000"] = np.deg2rad(np.array([-36,]))
bcg_table['sourceType']  = np.array(["galaxy",])
bcg_table['DiskHalfLightRadius'] = np.array([5.76,])
bcg_table['BulgeHalfLightRadius'] = np.array([5.76])
bcg_table['gmagVar'] = np.array([_gr + _ri + _imag,])
bcg_table['rmagVar'] = np.array([_ri + _imag,])
bcg_table['imagVar'] = np.array([_imag,])
bcg_table['zmagVar'] = np.array([_imag - _iz,])
bcg_table['disk_n'] = np.array([1.0,])
bcg_table['bulge_n'] = np.array([3.0,])
bcg_table['a_d'] = np.array([2.0,])
bcg_table['a_b'] = np.array([2.0,])
bcg_table['b_d'] = np.array([2.0,])
bcg_table['b_b'] = np.array([2.0,])
bcg_table['pa_disk'] = np.array([35.0,])
bcg_table['pa_bulge'] = np.array([35.0,])

np.random.seed(10)
angle = np.random.uniform(0, 2. * np.pi, size=len(gals))
table = pd.DataFrame()
table["raJ2000"] = np.deg2rad((10**gals["LOGR"] * np.cos(angle)).values / 60. + 64)
table["decJ2000"] = np.deg2rad((10**gals["LOGR"] * np.sin(angle)).values / 60. - 36)
table['sourceType'] = np.array(["galaxy",]*len(gals))
table['DiskHalfLightRadius'] = 10**gals["SIZE_DISK"].values
table['BulgeHalfLightRadius'] = 10**gals["SIZE_BULGE"].values
table["gmagVar"] = (gals["COLOR_G_R"] + gals["COLOR_R_I"] + gals["MAG_I"]).values
table["rmagVar"] = (gals["COLOR_R_I"] + gals["MAG_I"]).values
table["imagVar"] = gals["MAG_I"].values
table["zmagVar"] = (gals["MAG_I"] - gals["COLOR_I_Z"]).values
table['disk_n'] = np.array([1.0,]*len(gals))
table['bulge_n'] = np.array([3.0,]*len(gals))
# table['a_b'] = -1
table['a_d'] = 10**gals["SIZE_DISK"].values
table['a_b'] = (10**gals["SIZE_BULGE"].values) * 0.8
table['b_d'] = table["a_d"].values * 0.8
table['b_b'] = table["a_b"].values * 0.8
table['pa_disk'] = np.random.uniform(0, 360, size=len(gals))
table['pa_bulge'] = table["pa_disk"].values

table = pd.concat((bcg_table, table))

fio.write("test_cluster_fsi_v03.fits", table.to_records(), clobber=True)

In [166]:
table

,raJ2000,decJ2000,sourceType,DiskHalfLightRadius,BulgeHalfLightRadius,gmagVar,rmagVar,imagVar,zmagVar,disk_n,bulge_n,a_d,a_b,b_d,b_b,pa_disk,pa_bulge
0,1.117021,-0.628397,galaxy,0.833030,0.717590,20.861814,19.325012,18.813550,18.483820,1.0,3.0,0.833030,0.574072,0.666424,0.459258,304.766363,304.766363
1,1.117082,-0.628309,galaxy,0.970263,0.651274,21.655925,20.274712,19.809881,19.508440,1.0,3.0,0.970263,0.521019,0.776210,0.416815,350.298864,350.298864
2,1.116958,-0.628377,galaxy,0.694196,0.324470,23.351750,22.296008,21.960173,21.798609,1.0,3.0,0.694196,0.259576,0.555356,0.207661,75.931857,75.931857
3,1.117011,-0.628342,galaxy,0.260943,0.149901,24.090218,22.750172,22.361625,22.192145,1.0,3.0,0.260943,0.119921,0.208754,0.095937,179.720773,179.720773
4,1.116920,-0.628318,galaxy,0.742494,0.588211,24.365988,22.717601,21.679242,20.904513,1.0,3.0,0.742494,0.470568,0.593996,0.376455,161.271198,161.271198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1479,1.116240,-0.629650,galaxy,0.172537,0.091186,25.191791,24.229606,23.979161,23.850679,1.0,3.0,0.172537,0.072949,0.138029,0.058359,211.764334,211.764334
1480,1.117811,-0.629992,galaxy,0.118869,0.068388,25.840839,24.672814,24.428437,24.309805,1.0,3.0,0.118869,0.054711,0.095095,0.043769,188.744438,188.744438
1481,1.116083,-0.626099,galaxy,0.355662,0.174411,25.165342,24.807200,24.380021,23.822040,1.0,3.0,0.355662,0.139529,0.284529,0.111623,289.282762,289.282762
1482,1.116445,-0.625537,galaxy,0.378096,0.209330,24.491642,24.149937,23.836259,23.147904,1.0,3.0,0.378096,0.167464,0.302476,0.133971,34.104312,34.104312


In [171]:
table["a_d"] / table["b_d"]

0       1.25
1       1.25
2       1.25
3       1.25
4       1.25
        ... 
1479    1.25
1480    1.25
1481    1.25
1482    1.25
1483    1.25
Length: 1484, dtype: float64

In [138]:
table["b_d"]

0       0.833030
1       0.970263
2       0.694196
3       0.260943
4       0.742494
          ...   
1479    0.172537
1480    0.118869
1481    0.355662
1482    0.378096
1483    0.307374
Name: b_d, Length: 1484, dtype: float64

In [117]:
(table["a_d"] / table["b_d"])

0       1.0
0       1.0
1       1.0
2       1.0
3       1.0
       ... 
1479    1.0
1480    1.0
1481    1.0
1482    1.0
1483    1.0
Length: 1485, dtype: float64

In [105]:
(table["a_b"] / table["b_d"]).min()

0.3738650730257411

In [82]:
rarr = np.sqrt((table["RA"]**2  + table["DEC"]))

0       64.000000
0       64.000606
1       64.004101
2       63.996982
3       63.999990
          ...    
1479    63.955838
1480    64.045829
1481    63.946873
1482    63.967612
1483    64.042931
Name: RA, Length: 1485, dtype: float64

In [27]:
# Not sure if all of these are needed.
# table = Table()
# table['raJ2000'] = np.array([np.deg2rad(64.0), np.deg2rad(64.005)])
# table['decJ2000'] = np.array([np.deg2rad(-36.0), np.deg2rad(-36.005)])
# table['sourceType'] = np.array(["galaxy", "galaxy"])
# table['DiskHalfLightRadius'] = np.array([2.0, 2.0])
# table['BulgeHalfLightRadius'] = np.array([2.0, 2.0])
# table['umagVar'] = np.array([19.0, 18.0])
# table['gmagVar'] = np.array([19.0, 18.0])
# table['rmagVar'] = np.array([19.0, 18.0])
# table['imagVar'] = np.array([19.0, 18.0])
# table['zmagVar'] = np.array([19.0, 18.0])
# table['ymagVar'] = np.array([19.0, 18.0])
# table['disk_n'] = np.array([1.0, 1.0])
# table['bulge_n'] = np.array([3.0, 3.0])
# table['a_d'] = np.array([4.0, 4.0])
# table['a_b'] = np.array([4.0, 4.0])
# table['b_d'] = np.array([1.0, 1.0])
# table['b_b'] = np.array([1.0, 1.0])
# table['pa_disk'] = np.array([35.0, 35.0])
# table['pa_bulge'] = np.array([55.0, 55.0])

# table.write("fsi.fits", overwrite=True)

,COLOR_G_R,COLOR_I_Z,COLOR_R_I,GABS,LOGR,MAG_I,SIZE,SIZE_BULGE,SIZE_DISK,SIZE_MINOR,SIZE_MINOR_BULGE,SIZE_MINOR_DISK,STELLAR_MASS,Z
3214367,1.536802,0.329729,0.511463,0.014104,-0.565018,18.813550,-0.142989,-0.144123,-0.079340,-0.152745,-0.160880,-0.105685,11.145405,0.325
3375285,1.381213,0.301441,0.464830,0.059532,-0.605286,19.809881,-0.165175,-0.186236,-0.013111,-0.218608,-0.233215,-0.107522,10.496935,0.325
1153842,1.055742,0.161564,0.335835,0.031551,-0.566649,21.960173,-0.360137,-0.488826,-0.158518,-0.389342,-0.489438,-0.199872,9.303776,0.325
3261377,1.340046,0.169480,0.388547,0.418884,-1.084094,22.361625,-0.575251,-0.824194,-0.583455,-0.963365,-0.864020,-0.964242,9.116773,0.325
1947103,1.648387,0.774728,1.038359,0.225303,-0.508000,21.679242,-0.234666,-0.230467,-0.129307,-0.432516,-0.439854,-0.392179,11.364313,0.325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3245737,0.962185,0.128482,0.250445,0.087588,0.723338,23.979161,-0.951189,-1.040071,-0.763119,-1.027391,-1.044389,-1.034336,7.706235,0.325
2347330,1.168025,0.118632,0.244377,0.234383,0.804540,24.428437,-0.925445,-1.165018,-0.924933,-1.135317,-1.214350,-1.137436,8.321273,0.325
1524822,0.358142,0.557981,0.427179,0.156079,0.917547,24.380021,-0.481959,-0.758426,-0.448963,-0.621514,-0.804891,-0.602016,8.833749,0.325
117759,0.341704,0.688356,0.313678,0.085997,0.989302,23.836259,-0.492849,-0.679169,-0.422398,-0.566616,-0.683296,-0.541674,9.869985,0.325
